In [1]:
import json
import time
import os
import re

from modules.preprocessing_protocols import preprocess_text
from modules.extract_results import extract_results_to_txt_file, display_results_by_subdistribution,\
                                    display_results_by_named_entity, display_confusion_matrix
from contemporary_ner_training.conll_ner_importer import conll_to_ner_labelling  
from estnltk import Text
from estnltk.taggers import NerTagger
from estnltk.taggers import WordLevelNerTagger
from estnltk.converters import text_to_json
from estnltk.converters import json_to_text
from estnltk.layer_operations import flatten

from estnltk.taggers.estner.ner_trainer import NerTrainer
from estnltk.taggers.estner.model_storage_util import ModelStorageUtil

from estnltk.taggers import VabamorfCorpusTagger
vm_corpus_tagger = VabamorfCorpusTagger()

In [2]:
def find(name, path):
    for root, dirs, files in os.walk(path):
        if name in files:
            return os.path.join(root, name)

CONLL NER test:

In [3]:
'''
texts = []
for file in os.listdir('contemporary_ner_training'):
    if file.startswith('estner_split_1'):
        texts.append(conll_to_ner_labelling(os.path.join('contemporary_ner_training', file), 'gold_wordner'))               
'''

"\ntexts = []\nfor file in os.listdir('contemporary_ner_training'):\n    if file.startswith('estner_split_1'):\n        texts.append(conll_to_ner_labelling(os.path.join('contemporary_ner_training', file), 'gold_wordner'))               \n"

Flags:

In [4]:
use_vabamorfcorpustagger = False

---

In [5]:
files = {}

with open(os.path.join('..', 'data', 'divided_corpus.txt'), 'r', encoding = 'UTF-8') as f:
    txt = f.readlines()

for fileName in txt:
    file, subdistribution = fileName.split(":")
    files[file] = subdistribution.rstrip("\n")

In [6]:
def create_training_texts(filenames):
    print("Valmistan ette treenimistekste")
    
    # These files don't work because the protocols are written in a different language,
    # which the goldstandard didn't recognise, hence have no goldstandard tags.
    files_not_working = ['J2rva_Tyri_V22tsa_id22177_1911a.json', \
                         'J2rva_Tyri_V22tsa_id18538_1894a.json', \
                         'J2rva_Tyri_V22tsa_id22155_1911a.json', \
                         'Saare_Kihelkonna_Kotlandi_id18845_1865a.json', \
                         'P2rnu_Halliste_Abja_id257_1844a.json', \
                         'Saare_Kaarma_Loona_id7575_1899a.json', \
                         'J2rva_Tyri_V22tsa_id22266_1913a.json', \
                         'J2rva_Tyri_V22tsa_id22178_1912a.json']
    
    start = time.time()
    training_texts = []
    for filename in filenames:
        with open(os.path.join('..', 'data', 'vallakohtufailid-json-flattened', filename), 'r', encoding='UTF-8') as file:
            if filename in files_not_working:
                continue
            else:
                tagged_text = preprocess_text(json_to_text(file.read()))
                if use_vabamorfcorpustagger:
                    tagged_text.pop_layer('morph_analysis')
                    vm_corpus_tagger.tag([tagged_text])
                training_texts.append(tagged_text)
    print(f"Treenimistekstid ette valmistatud {time.time() - start} sekundiga.")
    return training_texts

In [7]:
def train_nertagger(training_texts, new_model_dir):
    print("Alustan NerTaggeri treenimist.")
    start = time.time()
    
    modelUtil = ModelStorageUtil( new_model_dir )
    nersettings = modelUtil.load_settings()
    trainer = NerTrainer(nersettings)
    trainer.train( training_texts, layer='gold_wordner', model_dir=new_model_dir )
    print(f"NerTagger treenitud {time.time() - start} sekundiga.")

In [8]:
def get_testing_and_training_subdistribution(subdistribution):
    training = []
    for y in [1, 2, 3, 4, 5]:
        if y == subdistribution:
            testing = y
        else:
            training.append(y)
    return testing, training

In [9]:
def tag_files(model_dir, testing_files, nertagger, use_vabamorfcorpustagger):
    removed_layers = ['sentences', 'morph_analysis', 'compound_tokens', 'ner', 'words', 'tokens']
    
    print("\n\nAlustan failide märgendamist.")
    start = time.time()
    
    for test_file in testing_files:
        with open(find(test_file.replace(".json", ".txt"), os.path.join('..', 'data', 'vallakohtufailid')), 'r', encoding='UTF-8') as f:
            text = f.read()
        
        if test_file == "Tartu_V6nnu_Ahja_id3502_1882a.json":
            text = text.replace('..', '. .')
        text = preprocess_text(Text(text))

        if use_vabamorfcorpustagger:
            text.pop_layer('morph_analysis')
            text = [text]
            vm_corpus_tagger.tag( text )
            text = text[0]
        nertagger.tag(text)
        text.add_layer(flatten(text['ner'], 'flat_ner'))

        for x in removed_layers:
            text.pop_layer(x)

        path = os.path.join(model_dir, 'vallakohtufailid-trained-nertagger')
        if not os.path.exists(path):
            os.mkdir(path)
            
        text_to_json(text, file=os.path.join(model_dir, 'vallakohtufailid-trained-nertagger', test_file))

            
        print(f'Märgendatud fail {test_file}')
    print(f"Failid märgendatud {time.time() - start} sekundiga.")

In [10]:
def train_model(model_directory):
    for subdistribution in [1, 2, 3, 4, 5]:
        testing, training = get_testing_and_training_subdistribution(subdistribution)
        
        # Get the filenames to be trained on from the files dictionary
        filenames = [key for key, value in files.items() if int(value) in training]

        # Create training_texts from the aforementioned filenames
        training_texts = create_training_texts(filenames)
        
        # Set up the trainer and training
        new_model_dir = os.path.join('models', model_directory)
        train_nertagger(training_texts, new_model_dir)
        
        # Set up the new trained nertagger and defining layers to be removed later on
        tagger = NerTagger(model_dir = new_model_dir)
        
        # Tag the files using the new nertagger
        testing_files = [key for key, value in files.items() if int(value) == testing]
        tag_files(new_model_dir, testing_files, tagger, use_vabamorfcorpustagger)
            
    # Get results of model
    extract_results_to_txt_file(model_directory, files)
    
    print(f"Mudel {model_directory} treenitud.")

To train the model the `model_directory` must contain a `settings.py` file

In [11]:
train_model(os.path.join('model_morph_with_lemmas_and_sentences_and_gazetteer_and_global_features',
                         'model_gaz_loc_variants'))

Valmistan ette treenimistekste
Treenimistekstid ette valmistatud 114.97801065444946 sekundiga.
Alustan NerTaggeri treenimist.
(!) Warning: Location of the new "settings.py" is the same one as the old one. Model's settings are not copied.


NotImplementedError: _change_layer method not implemented in NerGazetteerFeatureTagger